# Lab 3: Message authentication and Integrity protection
Laboratory session 3 of *Information Security*, AY 2024/25. In this notebook, we implement an authentication scheme and evaluate its weakness through a substitution and a forging attack.


In [1]:
# library imports
import numpy as np
import matplotlib.pyplot as plt
import random

## Task 1: Authentication scheme
Consider the following symmetric scheme for message authentication and integrity protection:
- the message $u$ is a sequence of $M$ bits
- the key $k$ is a sequence of $K$ bits
- the authentication tag $t$ is the binary representation of the integer product $s = s_u s_k$, where $s_u$ and
$s_k$ are the sum of the digits in the decimal (base 10) representation of $u$ and $k$, respectively.

The tag is transmitted along with the message $u$, and the receiver, which knows $k$, computes the tag
from the received message and checks whether it is identical to the received tag. If so, the message is
accepted as authentic.
Using a programming language of your choice, implement the above scheme, and the corresponding
verification so that the complexity of each is polynomial in both $M$ and $K$.

In [2]:
def computeDigitSum(x: int):
    # computes sum of digits of a given integer x
    s = 0
    while x > 0:
        s += x % 10
        x //= 10
    return s
    
def computeTag(u, k):
    # computes authentication tag given 2 bit sequences: message u and key k
    s_u = computeDigitSum(int(u, 2))
    s_k = computeDigitSum(int(k, 2))
    #t = bin(s_u*s_k)[2:]
    t = f'{s_u*s_k:b}'
    return t 

def authScheme(u, k, verbose=False):
    t = computeTag(u, k) # compute tag
    x = u+t # append tag to message
    
    if verbose: # verbose output for debugging
        print(f"Message u: {u}")
        print(f"Key k: {k}")
        print(f"Tag t: {t}")
        print(f"Sending x: {x}")

    return x

# length M of message is known
def checkReceived(x, k, M, verbose=False):
    # checks authenticity based on tag of received bitstring x, 
    # knowing key k and length of message M
    
    # split received bitstring into message u and tag t
    u = x[:M] # message
    t = x[M:] # tag
    # compute real tag t from u and key k
    t_check = computeTag(u, k)
    accept = t == t_check
    
    if verbose: # verbose output for debugging
        print(f"received tag: {t}")
        print(f"computed tag: {t_check}")
        if accept:
            print("Message accepted!")
        else: ("Message rejected!")
            
    return accept

In [3]:
u = '00000001' # message
k = '1111111' # key
M = len(u) # known length of message bitstring
x = authScheme(u, k, verbose=True)
accept = checkReceived(x, k, M, verbose=True)

Message u: 00000001
Key k: 1111111
Tag t: 1010
Sending x: 000000011010
received tag: 1010
computed tag: 1010
Message accepted!


Test with random keys/messages:

In [4]:
# check if authentic messages are accepted correctly
accepted = 0
M = 8
K = 8
for _ in range(1000):
    u = "".join(str(random.randint(0, 1)) for i in range(M)) # message
    k = "".join(str(random.randint(0, 1)) for i in range(K)) # key
    x = authScheme(u, k)
    accepted += checkReceived(x, k, M)
print(f"Correctly accepted: {accepted} of 100")

# check if random inauthentic messages are rejected
T = 4 # fake tag length
accepted = 0
for _ in range(1000):
    x = "".join(str(random.randint(0, 1)) for i in range(M+T)) # fake message
    k = "".join(str(random.randint(0, 1)) for i in range(K)) # key
    accepted += checkReceived(x, k, M)
print(f"Incorrectly accepted: {accepted} of 100")

Correctly accepted: 1000 of 100
Incorrectly accepted: 1 of 100
